# Konto Mieszkaniowe

## Podsumowanie

* na pierwsze mieszkanie (z wyjątkami) dla osób od 13 do 45 roku życia.
* oszczędzanie 3-10 lat, potem max 5 lat na zakup mieszkania/nieruchomości.
* wymaga regularnego (min. 11 miesięcznych rat każdego roku, każda od 500-2000 zł).
* państwo gwarantuje premię (po spełnieniu warunków) w wysokości inflacji lub wzrostu cen mieszkań (większy z tych wskaźników), `1% < premia < 15%`
* dodatkowo bank wypłaca odsetki w wysokści co najmniej 75% standardowej ofery lokaty banku.  Od tych odsetek nie płacimy podatku Belki.
* max. jedno konto na osobę, przy czym konto jest prowadzone w imieniu dziecka a nie rodzica w przypadku niepełnoletnich dzieci.

## Podstawa Prawna

* Ustawa z dnia 26 maja 2023 r. o pomocy państwa w oszczędzaniu na cele mieszkaniowe ([Dz.U. poz. 1114](https://dziennikustaw.gov.pl/DU/2023/1114))
* Ustawa z dnia 1 października 2021 r. o rodzinnym kredycie mieszkaniowym i bezpiecznym kredycie 2% ([Dz. U. 2021 r. poz. 2133](https://isap.sejm.gov.pl/isap.nsf/DocDetails.xsp?id=WDU20210002133))

## Materiały

* [PierwszeMieszkanie.gov.pl](https://pierwszemieszkanie.gov.pl/)
* [gov.pl: Ministerstwo Rozwoju i Technologii: Konto Mieszkaniowe](https://www.gov.pl/web/rozwoj-technologia/konto-mieszkaniowe)
* [gov.pl: Ministerstwo Rozwoju i Technologii: Program Pierwsze Mieszkanie](https://www.gov.pl/web/rozwoj-technologia/pierwsze-mieszkanie)
* [bgk.pl: Konto Mieszkaniowe / Program Pierwsze Mieszkanie: O produkcie](https://www.bgk.pl/osoby-prywatne/mieszkalnictwo/konto-mieszkaniowe/#c28296)
* [bgk.pl: Konto Mieszkaniowe / Program Pierwsze Mieszkanie: Lista banków prowadzących Konto Mieszkaniowe](https://www.bgk.pl/osoby-prywatne/mieszkalnictwo/konto-mieszkaniowe/#c28304)
* [Subiektywnie o Finansach: W pierwszym banku już dziś założysz Konto Mieszkaniowe!](https://subiektywnieofinansach.pl/od-dzis-zalozysz-konto-mieszkaniowe-w-pierwszym-banki-warto/)
* [Marcin Iwuć: Konto Mieszkaniowe - czy to się opłaca?](https://marciniwuc.com/konto-mieszkaniowe/)

## Kod do wyliczenia szacowanych zysków

## Test symulatora
Liczymy scenariusz ze stron rządowych [PierwszeMieszkanie.gov.pl](https://pierwszemieszkanie.gov.pl/), a konkretnie ten scenariusz:
<img src="https://www.gov.pl/photo/d64c123f-f6b6-4e32-982c-ae85680f408d" width="500"/>
Czyli:

* 3 lata oszczędzania
* po 1000zł miesięcznie
* przy 9.6% inflacji przez wszystkie 3 lata, wskaźnik wzrostu cen m2 nieistotny


In [ ]:
from konmiesz import *

df_inflacja1 = zalozenia_inflacji_i_wzrostu_m2(
    inflacja=[pct/100 for pct in [9.6] * 3], wzrost_m2=None)
df_konto1_styled, df_roczne1_styled, df_konto1, _ = symulacja_konta(
    data_startu='2024-01', ile_lat=3, ile_wplat=3*12, wysokosc_wplat=1000,
    premia=df_inflacja1)
rysuj_wykres_lokaty(df_konto1)
display(formatuj_inflacje(df_inflacja1))
display(df_roczne1_styled)
display(df_konto1_styled)

## Założenia inflacji i wzrostu cen nieruchomości

Prognozy inflacji oparte o [Polski Instytut Ekonomiczny: Struktura inflacji CPI – prognoza PIE](https://pie.net.pl/prognozy/) i długoterminowy cel inflacyjny NBP (2.5%).

Dla prostoty na razie nie uwzględniam wzrostu cen nieruchomości, który w niektórych latach może przekraczać inflację i zwiększyłby wysokość premii mieszkaniowej.

## Symulacja oszczędzania

In [ ]:
df_inflacja2 = zalozenia_inflacji_i_wzrostu_m2(
    inflacja=[pct/100 for pct in [7.6, 4.3] + [2.5] * 8], wzrost_m2=None)
df_konto2_styled, df_roczne2_styled, df_konto2, _ = symulacja_konta(
    data_startu='2024-01', ile_lat=10, ile_wplat=10*12, wysokosc_wplat=1000,
    premia=df_inflacja2)

rysuj_wykres_lokaty(df_konto2)
display(formatuj_inflacje(df_inflacja2))
display(df_roczne2_styled)
display(df_konto2_styled)